In [20]:
import numpy as np

In [21]:
# import time (обучение как процесс оптимизации здесь отсутствует)

In [22]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving data_thesis.csv to data_thesis (1).csv


In [23]:
df = pd.read_csv('data_thesis.csv', parse_dates=['TRANSACTION_DT'])

df['MONTH'] = df['TRANSACTION_DT'].dt.to_period('M')

def filter_data(df):
    user_counts = df['CUSTOMER_ID'].value_counts()
    item_counts = df['PRODUCT_ID'].value_counts()

    df_filtered = df[df['CUSTOMER_ID'].isin(user_counts[user_counts >= 10].index)]
    df_filtered = df_filtered[df_filtered['PRODUCT_ID'].isin(item_counts[item_counts >= 10].index)]

    total_months = df_filtered['MONTH'].nunique()

    user_monthly_counts = df_filtered.groupby(['CUSTOMER_ID', 'MONTH']).size().groupby('CUSTOMER_ID').mean()

    valid_users = user_monthly_counts[user_monthly_counts >= 1].index
    df_filtered = df_filtered[df_filtered['CUSTOMER_ID'].isin(valid_users)]

    return df_filtered

prev_shape = None
current_df = df.copy()

while prev_shape != current_df.shape:
    prev_shape = current_df.shape
    current_df = filter_data(current_df)

In [24]:
current_df = current_df.drop(columns=['TRANSACTION_DT', 'ASSET'])
current_df

,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,SALES_PRICE,MONTH
0,1104905,45-49,115,110411,4710199010372,2,30,2000-11
1,418683,45-49,115,120107,4710857472535,1,46,2000-11
2,1057331,35-39,115,100407,4710043654103,2,166,2000-11
3,1849332,45-49,Others,120108,4710126092129,1,38,2000-11
4,1981995,50-54,115,100205,4710176021445,1,18,2000-11
...,...,...,...,...,...,...,...,...
817734,234658,45-49,Unknown,530104,4710168182031,1,149,2001-02
817735,556941,35-39,115,712901,8888021800401,1,150,2001-02
817737,57486,40-44,115,530209,4710731060124,1,55,2001-02
817738,733526,>65,Unknown,510539,4716340052307,1,115,2001-02


In [25]:
for col in current_df.columns:
    unique_count = current_df[col].nunique()
    print(f"Column '{col}': {unique_count} unique values")

Column 'CUSTOMER_ID': 19745 unique values
Column 'AGE_GROUP': 10 unique values
Column 'PIN_CODE': 8 unique values
Column 'PRODUCT_SUBCLASS': 1342 unique values
Column 'PRODUCT_ID': 10721 unique values
Column 'AMOUNT': 80 unique values
Column 'SALES_PRICE': 1879 unique values
Column 'MONTH': 4 unique values


In [26]:
target_month = pd.Period('2001-02', freq='M')

In [27]:
past_df = current_df.dropna(subset=['CUSTOMER_ID', 'PRODUCT_ID', 'AMOUNT', 'SALES_PRICE'])
past_df['AMOUNT'] = past_df['AMOUNT'].astype(float)
past_df['SALES_PRICE'] = past_df['SALES_PRICE'].astype(float)
future_df = current_df[current_df['MONTH'] == target_month]

In [28]:
user_product_counts = past_df.groupby(
    ['CUSTOMER_ID', 'PRODUCT_ID', 'PRODUCT_SUBCLASS', 'AGE_GROUP'],
    as_index=False
).agg(
    total_amount=('AMOUNT', 'sum'),
    avg_sales_price=('SALES_PRICE', 'mean')
)

user_product_counts = user_product_counts.sort_values(by='total_amount', ascending=False)

In [29]:
top_n = 50

In [30]:
user_product_counts

,CUSTOMER_ID,PRODUCT_ID,PRODUCT_SUBCLASS,AGE_GROUP,total_amount,avg_sales_price
508175,1970777,4710421090059,100508,35-39,1331.0,7784.0
582894,2120829,4710421090059,100508,25-29,1206.0,7443.0
584721,2123851,4710421090059,100508,25-29,1206.0,7443.0
335634,1622362,4711588210441,100516,<25,800.0,444000.0
591257,2134048,4710421090059,100508,55-59,720.0,8424.0
...,...,...,...,...,...,...
618629,2179605,4711080010112,130206,25-29,1.0,40.0
618628,2179605,4710868506182,110133,25-29,1.0,96.0
618627,2179605,4710735405006,590312,25-29,1.0,139.0
618626,2179605,4710731041505,590311,25-29,1.0,119.0


In [31]:
def weighted_total_amount(df, target_month, decay=0.5):
    df['months_diff'] = (target_month - df['MONTH']).apply(lambda x: x.n)
    df['time_weight'] = np.exp(-decay * df['months_diff'])
    df['weighted_amount'] = df['AMOUNT'] * df['time_weight']
    agg = df.groupby(['CUSTOMER_ID', 'PRODUCT_ID', 'PRODUCT_SUBCLASS', 'AGE_GROUP'], as_index=False).agg(
        weighted_total_amount=('weighted_amount', 'sum'),
        avg_sales_price=('SALES_PRICE', 'mean')
    )
    return agg

weighted_user_product = weighted_total_amount(past_df, target_month)

In [32]:
def get_top_n_weighted(df, n=top_n):
    df = df.sort_values(by=['weighted_total_amount', 'avg_sales_price'], ascending=[False, True])
    return df[['PRODUCT_ID', 'PRODUCT_SUBCLASS', 'avg_sales_price', 'AGE_GROUP']].head(n).to_dict('records')

In [33]:
recommendations_weighted = weighted_user_product.groupby('CUSTOMER_ID').apply(get_top_n_weighted).reset_index()
recommendations_weighted.columns = ['CUSTOMER_ID', 'TopPersonal_Recommendations']
recommendations_weighted = recommendations_weighted.set_index('CUSTOMER_ID')['TopPersonal_Recommendations'].to_dict()

/tmp/ipython-input-33-3314025494.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recommendations_weighted = weighted_user_product.groupby('CUSTOMER_ID').apply(get_top_n_weighted).reset_index()


In [34]:
recommendations_weighted

Output hidden; open in https://colab.research.google.com to view.

In [35]:
def recall_at_k(true_items, pred_items, k):
    true_set = set(true_items)
    pred_k = pred_items[:k]
    hits = len(set(pred_k) & true_set)
    if len(true_set) == 0:
        return 0.0
    return hits / len(true_set)

def precision_at_k(true_items, pred_items, k):
    true_set = set(true_items)
    pred_k = pred_items[:k]
    hits = len(set(pred_k) & true_set)
    return hits / k

def average_precision_at_k(true_items, pred_items, k):
    true_set = set(true_items)
    pred_k = pred_items[:k]
    hits = 0
    sum_precisions = 0.0
    for i, p in enumerate(pred_k, start=1):
        if p in true_set:
            hits += 1
            sum_precisions += hits / i
    if hits == 0:
        return 0.0
    return sum_precisions / hits

def dcg_at_k(true_items, pred_items, k):
    true_set = set(true_items)
    pred_k = pred_items[:k]
    dcg = 0.0
    for i, p in enumerate(pred_k, start=1):
        if p in true_set:
            dcg += 1 / np.log2(i + 1)
    return dcg

def idcg_at_k(true_items, k):
    ideal_hits = min(len(true_items), k)
    idcg = sum(1 / np.log2(i + 1) for i in range(1, ideal_hits + 1))
    return idcg

def ndcg_at_k(true_items, pred_items, k):
    idcg = idcg_at_k(true_items, k)
    if idcg == 0:
        return 0.0
    dcg = dcg_at_k(true_items, pred_items, k)
    return dcg / idcg

In [36]:
ground_truth = future_df.groupby('CUSTOMER_ID')['PRODUCT_ID'].apply(list).to_dict()

In [37]:
Ks = [10, 20, 50]

results = {k: {'Recall': [], 'NDCG': [], 'MAP': []} for k in Ks}

for user_id in ground_truth:
    true_items = ground_truth[user_id]
    pred_items_dicts = recommendations_weighted.get(user_id, [])
    pred_items = [item['PRODUCT_ID'] for item in pred_items_dicts]
    for k in Ks:
        results[k]['Recall'].append(recall_at_k(true_items, pred_items, k))
        results[k]['NDCG'].append(ndcg_at_k(true_items, pred_items, k))
        results[k]['MAP'].append(average_precision_at_k(true_items, pred_items, k))

for k in Ks:
    recall = np.mean(results[k]['Recall'])
    ndcg = np.mean(results[k]['NDCG'])
    map_score = np.mean(results[k]['MAP'])
    print(f"Metrics @ {k}: Recall={recall:.4f}, NDCG={ndcg:.4f}, MAP={map_score:.4f}")

Metrics @ 10: Recall=0.7064, NDCG=0.8434, MAP=0.8490
Metrics @ 20: Recall=0.9005, NDCG=0.8728, MAP=0.8353
Metrics @ 50: Recall=0.9763, NDCG=0.8891, MAP=0.8310


*   Recall очень высокий при большом числе рекомендаций — система хорошо покрывает релевантные товары.
*   Высокие значения NDCG говорят о том, что релевантные объекты расположены достаточно близко к началу списка.
*   Значения MAP на уровне около 0.83–0.84 — очень хорошие

In [38]:
def extract_product_ids(recs):
    return [rec['PRODUCT_ID'] for rec in recs]

recommendations_simple = {user: extract_product_ids(recs)
                          for user, recs in recommendations_weighted.items()}

true_purchases = future_df.groupby('CUSTOMER_ID')['PRODUCT_ID'].apply(set).to_dict()

def mean_reciprocal_rank(recommended, actual):
    for rank, product_id in enumerate(recommended, start=1):
        if product_id in actual:
            return 1.0 / rank
    return 0.0

def hit_rate(recommended, actual):
    return int(any(p in actual for p in recommended))

for K in [10, 20, 50]:
    mrrs = []
    hits = []

    for user, recs in recommendations_simple.items():
        actual = true_purchases.get(user, set())
        if not actual:
            continue
        recs_at_k = recs[:K]
        mrrs.append(mean_reciprocal_rank(recs_at_k, actual))
        hits.append(hit_rate(recs_at_k, actual))

    mean_mrr = np.mean(mrrs) if mrrs else 0
    mean_hit_rate = np.mean(hits) if hits else 0

    print(f"MRR@{K}: {mean_mrr:.4f}")
    print(f"HitRate@{K}: {mean_hit_rate:.4f}")



MRR@10: 0.9094
HitRate@10: 0.9990
MRR@20: 0.9095
HitRate@20: 1.0000
MRR@50: 0.9095
HitRate@50: 1.0000


Значение MRR@K очень близко к 1, что означает, что в среднем первый релевантный товар находится почти на первом месте в рекомендациях.
Значение HR@K = 1.0 (100%) указывает на очень высокую полноту рекомендаций